# Mavlink Camera
> Mavlink Camera Component for sending commands to a camera on a companion computer or GCS
> The server is on the companion computer and the client is on the ground station PC.

In [ ]:
#| default_exp mavlink.camera

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time, os, sys

from UAV.logging import logging
from UAV.mavlink.mavcom import MAVCom
from UAV.mavlink.component import Component, mavutil



# from UAV.imports import *   # TODO why is this relative import on nbdev_export?


In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
from viewsheen_sdk import GST_Video

from viewsheen_sdk.gimbal_cntrl import pan_tilt, snapshot,  zoom, VS_IP_ADDRESS, VS_PORT, KeyReleaseThread

In [ ]:
import socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Connect to viewsheen_sdk gimbal
sock.connect((VS_IP_ADDRESS, VS_PORT))

In [ ]:

gimbal_speed = 40
data = pan_tilt(gimbal_speed)
sock.sendall(data)

In [ ]:
assert False, "Stop here"

AssertionError: Stop here

In [ ]:

sock.close()

In [ ]:
#| export
class Camera(Component):
    """Create a mavlink Camera Component using a test GSTREAMER pipeline"""

    def __init__(self, mav_connection,  # MavLinkBase connection
                 source_component,  # used for component indication
                 mav_type,  # used for heartbeat MAV_TYPE indication
                 debug):  # logging level
        
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)
        

    def start_streaming(self, frame_rate=30, resolution=0, bayer_pattern=0, video_stream_id=0):
        """Start streaming video"""
        
        # """Send a command and wait for an ack"""
        # 
        # self.send_command(target_system, target_component, cmd_id)
        # self.wait_ack(target_system=target_system, target_component=target_component, cmd_id=cmd_id)



In [ ]:
# | hide
# assert False, "Stop here"

In [ ]:
show_doc(Component)

#### Test locally using UDP ports

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:
from UAV.mavlink.mavcom import MAVCom
from UAV.mavlink.component import Component, mavutil
import time

MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA

class Cam1(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)

class Cam2(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)
class Cli(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("udpin:localhost:14445", source_system=111, debug=False) as client:
    with MAVCom("udpout:localhost:14445", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(Component.set_source_compenent)

In [ ]:
show_doc(Component.send_heartbeat)

In [ ]:
show_doc(Component.on_command_rcvd)

In [ ]:
show_doc(Component.send_command)

In [ ]:
show_doc(Component.wait_heartbeat)

In [ ]:
show_doc(Component.wait_ack)

In [ ]:
show_doc(Component.send_ping)

#### Test with Serial ports
Test using a Pixhawk connected via telemetry 2 and USB serial ports.
CamClient is set to udpin:localhost:14445 and CamServer is set to udpout:localhost:14435 udpin is so that the client can receive UDP from the mavproxy server at localhost:14445
mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445 mavproxy.py --master=/dev/ttyACM3 --baudrate 57600 --out udpout:localhost:14435

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("/dev/ttyACM0", source_system=111, debug=False) as client:
    with MAVCom("/dev/ttyUSB0", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

> For debugging help see http://localhost:3000/tutorials/mavlink_doc&debug.html and http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()